In [1]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
import itertools
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [5]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [6]:
url = 'https://www.beckershospitalreview.com/virtual-conferences.html'

In [7]:
driver.get(url)

In [8]:
soup = bs(driver.page_source,'lxml')

In [9]:
end = 'Past Virtual Events'

In [10]:
start = 'Upcoming Virtual Events'

In [11]:
lst = soup.find_all('li')

In [12]:
for i,j in enumerate(lst):
    if start in j.text:
        tag_start = i
    if end in j.text:
        tag_end = i

In [13]:
events = lst[tag_start+1:tag_end]


In [17]:
beker=[]
for k in events:
    link = k.find('a')['href']
    tit = k.text
    #data.append([tit,link])
    driver.get(link)
    time.sleep(2)
    try:
        time.sleep(5)
        soapy=bs(driver.page_source,'lxml')
        title=' '.join(soapy.find('div',{'id':'60d4fd73a6129'}).text.split())
    except:
        try:
            time.sleep(5)
            soapy=bs(driver.page_source,'lxml')
            title=' '.join(soapy.find('div',{'id':'60d4fd73a6129'}).text.split())
        except:
            try:
                time.sleep(5)
                soapy=bs(driver.page_source,'lxml')
                title=' '.join(soapy.find('div',{'id':'6192834f9fbca'}).text.split())
            except:
                title=''
    #***************
    try:
        dati=' '.join(soapy.find('div',{'id':'60d4fd73a6130'}).text.split())
        if '//' not in dati:
            dati=' '.join(soapy.find('div',{'id':'61fc46521e6f4'}).text.split())
        dat,tim=dati.split('//')

    except:
        dati=' '.join(soapy.find('div',{'id':'6192834f9fbcc'}).text.split())
        dat,tim=dati.split('|')
    dat=dat.strip().replace(',','').replace('–','-')
    #***************************
    try:
        event_info=' '.join(soapy.find('div',{'id':'60d4fda0a6131'}).text.split())
    except:
        event_info=' '.join(soapy.find('div',{'id':'61928acc9fbfd'}).text.split())
    #################################
    
    #***************************
    cb=tim.strip().replace(',','').replace('–','-')
    sd,edm=cb.split('-')
    sd=sd.strip()
    eru=edm.split(' ')
    emu=[pp.strip() for pp in eru]
    zn=emu[-1]
    ed=' '.join(emu[:-1])
    if any(c.isalpha() for c in sd)==False:
        pa=re.search('[A-P]{2}',ed).group()
        star=sd+' '+pa.strip()
        endi=ed.strip()
    else:
        star=sd.strip()
        endi=ed.strip()
    st_=star.replace(' ',':00')
    et_=endi.replace(' ',':00')
    time_st=[{'type': 'general',
             'start_time':st_,
             'end_time': et_,
             'timezone': zn,
             'days': 'all'}]
    time_=json.dumps(time_st)
      #**************************************************
    mad_=dat.split(' ')
    mad=mad_[1]+' '+mad_[0]+' '+mad_[2]#re.search('\W+\d{4}',ad).group()

    if '-' in mad:
        st=mad.split('-')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('-')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif 'and'in mad:
        st=mad.split('and')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('and')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif '&' in mad:
        st=mad.split('&')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('&')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    else:
        st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
    #print(date__,st,en)
    if any(c.isalpha() for c in st)==False:
    #      print('leg')
        pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en
    #                        02        April      2022
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
     #   print('awujale')
        start=st
        end=en

    #*                        April       02      2022
    elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
    #       print('ala')
        pa=re.search('(\d{4})',st).group()
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()
        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba     

    #                      02     2022
    elif re.search('\d{1,2}\s+\d{4}',en):
    #      print('kum')
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',st).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    #                      02         April
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
    #        print('is')
        pa=re.search('(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en

    #*                      April           02
    elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
    #      print('bad')
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    else:
    #     print('shik')
        start=end=''
    if start=='':
        start_date=end_date=''
    else:
        pick=[start, end]
        spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
        date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
        start_date=date_[0]
        end_date=date_[1]
    #***********************************************
    try:
        sob=soapy.find_all('a',{'data-container-id':re.compile('61f2ca233def.+')})
        sub=soapy.find_all('a',{'data-container-id':re.compile('617829d8862e.+')})
    except:
        sponsors=''
    seb=sob+sub
    fri=[]
    for m in seb:
        dum=m['href']
        du=re.search('\.\w+\/?',dum).group().replace('.','')
        if 'li' in du or 'hub' in du:
            du=m.find('img')['src'].split('/')[-1].split('-')[0]
        elif 'com/' in du:
            du=re.search('/+\w+\/?',dum).group().replace('.','').replace('/','')
        elif '/' in du:
            try:
                du=m.find('img')['src'].split('/')[-1].split('-')[0]
            except:
                du=re.search('/+\w+\/?',dum).group().replace('.','').replace('/','')
        fri.append(du)
    if fri=='' or fri==[] or fri==['']:
        sponsors=''
    else:
        sponsors=json.dumps(fri)
    try:
        sn=soapy.find_all('div',attrs={'data-container-id':re.compile('61786e8ac9be9,5f5fc5a10bbf.+'),'class':'component page-object group'})
    except:
        sn=soapy.find_all('div',attrs={'data-container-id':re.compile('6256fa2a81c40,61928eb79fc1.+'),'class':'component page-object group'})
    num=0
    breaker=True
    while breaker:
        try:
            element=driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[6]/div[2]/div/button[2]')
            driver.execute_script("return arguments[0].scrollIntoView();", element)
            #driver.execute_script("window.scrollBy(0, -200);")
            time.sleep(1)
            driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[6]/div[2]/div/button[2]').click()
            time.sleep(1)
        except:
            try:
                element=driver.find_element_by_xpath("/html/body/div[1]/div[1]/div[7]/div[2]/div/button[2]")
                driver.execute_script("return arguments[0].scrollIntoView();", element)
                #driver.execute_script("window.scrollBy(0, -100);")
                time.sleep(1)
                driver.find_element_by_xpath("/html/body/div[1]/div[1]/div[7]/div[2]/div/button[2]").click()#number1
                time.sleep(1)
            except:# ElementNotInteractableException:
                pass
        time.sleep(1)
        if num==10:
            breaker=False
        num+=1
        #****************
    soapy=bs(driver.page_source,'lxml')
    try:
        sn=soapy.find_all('div',attrs={'data-container-id':re.compile('61786e8ac9be9,5f5fc5a10bbf.+'),'class':'component page-object group'})
    except:
        sn=soapy.find_all('div',attrs={'data-container-id':re.compile('6256fa2a81c40,61928eb79fc1.+'),'class':'component page-object group'})
    time.sleep(1)
    sp=[a.find_all('div',{'class':'component page-object text'}) for a in sn]
    b=[]
    for c in sp:
        for e in c:
            d=' '.join(e.get_text(separator='$').split())
            b.append(d)
    if '' in b:
        f=[a.split('$') for a in b]
        b=[h for g in f for h in g]
        try:
            while True:
                b.remove('')
                #asa.remove('\xa0')
        except ValueError:
            pass
    def mygrouper(n, iterable):
        args = [iter(iterable)] * n
        return ([e for e in t if e != None] for t in itertools.zip_longest(*args))
    g=list(mygrouper(2,b))
    qq=[q+[""] for q in g]
    qqqq=[]
    for qqq in qq:
        emm={"name":qqq[0],
             "title":qqq[1],
             "link":qqq[2]}
        qqqq.append(emm)
    Speakerlist=json.dumps(qqqq)
    
    org_pro=''
    org_web='https://www.beckershospitalreview.com/'
    org_name="Becker's Healthcare"
    
    mass=['becker@beckershealthcare.com']
    mail_=json.dumps(mass)
    logo=''
    
    agendalist=''
    type_=''
    category=''
    
    event_web=link

    ticket_list=''
    on_off='1'
    city=''
    venue=''
    country=''
    googlePlaceUrl=''
        
        
    beker.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                 org_pro,org_name,org_web,logo,sponsors,agendalist,
                 type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                 Speakerlist,on_off])
print('Done')

Done


In [18]:
beker_dict=beker


beker_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=beker_dict)
beker_df.to_csv("beker.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [19]:
beker_df

,scrappedUrl,eventname,startdate,enddate,timing,eventinfo,ticketlist,orgProfile,orgName,orgWeb,...,type,category,city,country,venue,event_website,googlePlaceUrl,ContactMail,Speakerlist,online_event
0,https://events.beckershospitalreview.com/june-...,Hospital Operations Virtual Summit,2022-06-07,2022-06-08,"[{""type"": ""general"", ""start_time"": ""10:30am"", ...",The past year has been historically disruptive...,,,Becker's Healthcare,https://www.beckershospitalreview.com/,...,,,,,,https://events.beckershospitalreview.com/june-...,,"[""becker@beckershealthcare.com""]",[],1
1,https://events.beckershospitalreview.com/nursi...,NURSING LEADERSHIP VIRTUAL FORUM,2022-05-10,2022-05-10,"[{""type"": ""general"", ""start_time"": ""1:00PM"", ""...",Nurse leaders have a difficult task of putting...,,,Becker's Healthcare,https://www.beckershospitalreview.com/,...,,,,,,https://events.beckershospitalreview.com/nursi...,,"[""becker@beckershealthcare.com""]","[{""name"": ""Karen Grimley"", ""title"": ""PhD, MBA,...",1
2,https://events.beckershospitalreview.com/cardi...,CARDIOLOGY VIRTUAL FORUM,2022-05-18,2022-05-18,"[{""type"": ""general"", ""start_time"": ""10:00AM"", ...",Cardiologists are on the front lines of fighti...,,,Becker's Healthcare,https://www.beckershospitalreview.com/,...,,,,,,https://events.beckershospitalreview.com/cardi...,,"[""becker@beckershealthcare.com""]","[{""name"": ""Angel Le\u00f3n"", ""title"": ""MD, Pro...",1
3,https://events.beckershospitalreview.com/data-...,DATA + INNOVATION VIRTUAL EVENT,2022-05-25,2022-05-26,"[{""type"": ""general"", ""start_time"": ""1:00PM"", ""...",Leveraging data and technological advancements...,,,Becker's Healthcare,https://www.beckershospitalreview.com/,...,,,,,,https://events.beckershospitalreview.com/data-...,,"[""becker@beckershealthcare.com""]","[{""name"": ""Jonathan Griffin"", ""title"": ""MD, MH...",1
4,https://events.beckershospitalreview.com/shift...,THE SHIFT TO DIGITAL VIRTUAL EVENT,2022-06-01,2022-06-02,"[{""type"": ""general"", ""start_time"": ""10:00AM"", ...",The pandemic exacerbated the need for health s...,,,Becker's Healthcare,https://www.beckershospitalreview.com/,...,,,,,,https://events.beckershospitalreview.com/shift...,,"[""becker@beckershealthcare.com""]","[{""name"": ""JD Whitlock"", ""title"": ""Chief Infor...",1
5,https://events.beckershospitalreview.com/labor...,LABORATORY VIRTUAL FORUM,2022-06-14,2022-06-14,"[{""type"": ""general"", ""start_time"": ""10:00AM"", ...",Laboratories are an essential component to the...,,,Becker's Healthcare,https://www.beckershospitalreview.com/,...,,,,,,https://events.beckershospitalreview.com/labor...,,"[""becker@beckershealthcare.com""]","[{""name"": ""James M Crawford"", ""title"": "" MD, P...",1
6,https://events.beckershospitalreview.com/oncol...,ONCOLOGY VIRTUAL SUMMIT,2022-07-12,2022-07-12,"[{""type"": ""general"", ""start_time"": ""1:00PM"", ""...",Oncologists across the U.S. are preparing for ...,,,Becker's Healthcare,https://www.beckershospitalreview.com/,...,,,,,,https://events.beckershospitalreview.com/oncol...,,"[""becker@beckershealthcare.com""]","[{""name"": ""JoAnn Lovins"", ""title"": ""MS, RN, NE...",1
7,https://events.beckershospitalreview.com/hrtal...,HR + TALENT VIRTUAL SUMMIT,2022-07-14,2022-07-14,"[{""type"": ""general"", ""start_time"": ""10:00AM"", ...",At the crux of the healthcare system is the pe...,,,Becker's Healthcare,https://www.beckershospitalreview.com/,...,,,,,,https://events.beckershospitalreview.com/hrtal...,,"[""becker@beckershealthcare.com""]","[{""name"": ""Gregory Till"", ""title"": ""PhD, MA, E...",1
